In [ ]:
from fastai.vision.all import * 
import os
import tifffile as tiff

In [ ]:
#Replace with the raw data directory
src = '/home/susa03/post_project/raw_data'
images = 'images'   ### Hyperstack 7 channels (bse_en0, bse_en1,bse_ex0,bse_x1, normal map) into one iamge
masks = 'masks'     ### Store ground truth image

In [ ]:
#Replace with the arrange data directory
#dst = '/home/susa03/post_project/data_arrange/Cylinder'
dst = '/home/susa03/post_project/data_arrange/Sphere'
os.makedirs(os.path.join(dst,images), exist_ok=True)  # create destination folder if not exists
os.makedirs(os.path.join(dst,masks), exist_ok=True)  # create destination folder if not exists

# Get the full path of the created directory
image_path = os.path.join(dst, images)
masks_path = os.path.join(dst, masks)

In [ ]:
image_files = get_image_files(src)
image_height = 850
image_width = 850

In [ ]:
def return_cat(filename:str,category:str):
    value = str(category) in str(filename)
    return value

def make_category_list(filenames):
 
    # Iterate over the images in groups of 8
    group_size = 8
    for i in range(0, len(image_files), group_size):
        image_group = image_files[i:i+group_size]
        current_image= np.zeros((image_height, image_width,7)) 
        current_mask = np.zeros((image_height, image_width,1)) 
    
        # Process the current group of images
        for file in image_group:
            if return_cat(file,category='entry_hf_0'):
                current_image_address = os.path.join(src,file)
                bse_en0 = tiff.imread(current_image_address)
                current_image[:,:,0] = bse_en0
            elif return_cat(file,category='entry_hf_1'):
                current_image_address = os.path.join(src,file)
                bse_en1 = tiff.imread(current_image_address)
                current_image[:,:,1] = bse_en1
            elif return_cat(file,category='exit_hf_0'):
                current_image_address = os.path.join(src,file)
                bse_ex0 = tiff.imread(current_image_address)
                current_image[:,:,2] = bse_ex0
            elif return_cat(file,category='exit_hf_1'):
                current_image_address = os.path.join(src,file)
                bse_ex1 = tiff.imread(current_image_address)
                current_image[:,:,3] = bse_ex1
            elif return_cat(file,category='normal'):
                current_image_address = os.path.join(src,file)
                normal = tiff.imread(current_image_address)
                current_image[:,:,4:7] = normal
            elif return_cat(file,category='gt'):
                current_mask_address = os.path.join(src,file)
                current_mask = tiff.imread(current_mask_address)
            else: pass

        current_image = current_image.astype(np.uint8)
        current_image_name = image_path +'\\'+'exp'+ str(i%7)+'.tif'          
        tiff.imwrite(current_image_name, np.transpose(current_image,(2,0,1)))

        current_mask = current_mask.astype(np.uint8)
        current_mask_name = masks_path +'\\'+'exp'+ str(i%7)+'.tif'          
        tiff.imwrite(current_mask_name,current_mask)

make_category_list(image_files)